# Convert raw txt output to JSON

Columns in txt are:

```bifurcation ID, number of bounces, pr, ptheta, pphi, energy, rocking number, in phase/out of phase, period```

In [15]:
import math

# Returns a set of all prime factors of n
def primeFactors(n):
    if (n == 0):
        return set([])
    factors = []

    # Print the number of twos that divide n 
    while n % 2 == 0:
        factors.append(2)
        n = n // 2
          
    # n must be odd at this point 
    # so a skip of 2 (i = i + 2) can be used 
    for i in range(3, int(math.sqrt(n))+1, 2): 
        # while i divides n, print i and divide n
        while n % i == 0:
            factors.append(i)
            n = n // i
              
    # Condition if n is a prime
    # number greater than 2
    if n > 2:
        factors.append(n)
    
    return set(factors)

# Returns a set of all factors of n
def factors(n):
    if (n == 0):
        return set([])
    
    factors = []
    for i in range(1, n//2+1):
        if n % i == 0:
            factors.append(i)
    return set(factors)


def is_unique(m, n):
    return len(primeFactors(m).intersection(primeFactors(n))) == 0

print(primeFactors(0))
print(primeFactors(98))
print(factors(98))

print(primeFactors(74))
print(factors(74))

print(primeFactors(7))

print(primeFactors(6).intersection(primeFactors(4)))
print(primeFactors(4).intersection(primeFactors(6)))
print(primeFactors(3).intersection(primeFactors(3)))
print(primeFactors(3).intersection(primeFactors(1)))
print(primeFactors(3).intersection(primeFactors(2)))

print(is_unique(1, 1) == True)
print(is_unique(2, 1) == True)
print(is_unique(3, 1) == True)
print(is_unique(4, 1) == True)
print(is_unique(5, 1) == True)
print(is_unique(6, 1) == True)
print(is_unique(1, 2) == True)
print(is_unique(2, 2) == False)
print(is_unique(3, 2) == True)
print(is_unique(4, 2) == False)
print(is_unique(5, 2) == True)
print(is_unique(6, 2) == False)
print(is_unique(1, 3) == True)
print(is_unique(2, 3) == True)
print(is_unique(3, 3) == False)
print(is_unique(4, 3) == True)
print(is_unique(5, 3) == True)
print(is_unique(6, 3) == False)


set()
{2, 7}
{1, 2, 7, 14, 49}
{2, 37}
{1, 2, 37}
{7}
{2}
{2}
{3}
set()
set()
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


# Convert to json

Use this one!


In [29]:
import json
import re
import os
from os import listdir
from os.path import isfile, join

def getRockingNumber(s):
    if s['phase'] == 1:
        # In-phase
        return s['theta_crossings']
    else:
        # Out-of-phase
        return s['beta_crossings']

def getPhase(s):
    if s['phase'] == 1:
        # In-phase
        return 1
    else:
        # Out-of-phase
        return 2

def convert():
    data = []
    bifurcations = []
        
    mypath = './states'
    files = [mypath+'/'+f for f in listdir(mypath) if isfile(join(mypath, f)) and os.path.splitext(f)[1] == '.txt']
    files = sorted(files)
#     for num_bounces in range(1, 1000):
#         fn = './states{:03d}.txt'.format(num_bounces)
#     files = [mypath+'/states001.txt']
    for fn in files:
        exists = os.path.isfile(fn)
        if exists:
            with open(fn, 'r') as infile:
                print('reading {}'.format(fn))
                cur_mode = -1
                lines = [line for line in infile]
                for i, line in enumerate(lines):
                    line = line.strip()
                    (mode_id, n, pr, ptheta, pphi, E, ptheta_rocking, pphi_rocking, phase, period,
                    theta_crossings, phi_crossings, beta_crossings) = line.split(' ')
                    is_bifurcation = mode_id != cur_mode
                    state = { 'id':int(mode_id), 'bifurcation':is_bifurcation, 'n':int(n), 'pr':float(pr),
                             'ptheta':float(ptheta), 'pphi':float(pphi), 'E':float(E),
                             'ptheta_rocking':int(ptheta_rocking), 'pphi_rocking':int(pphi_rocking),
                             'phase':int(phase), 'period':float(period), 'theta_crossings':int(theta_crossings),
                             'phi_crossings':int(phi_crossings), 'beta_crossings':int(beta_crossings) }
                    data.append(state)
                    if is_bifurcation:
#                         bifurcations.append(state)
                        s = state
                        # bifurcations stores n, rocking, phase, E, period
                        bifurcations.append((s['n'], getRockingNumber(s), getPhase(s), s['E'], s['period']))
                    cur_mode = mode_id

    with open('./states_all.json', 'w') as outfile:  
        json.dump(data, outfile)

    bifurcations = list(set(bifurcations))
#     bifurcations.sort(key=lambda s: (s['n'], getRockingNumber(s), getPhase(s), s['E']))
    bifurcations.sort()
    duplicates = []
    for i, b in enumerate(bifurcations):
        if i < len(bifurcations)-1:
            c = bifurcations[i+1]
            if b[0] == c[0] and b[1] == c[1] and b[2] == c[2]:
                duplicates.append(b)
                duplicates.append(c)
    with open('./bifurcations.txt', 'w') as outfile:
        for s in bifurcations:
#             outfile.write('{}, {}, {}, {}, {}\n'.format(s['n'], getRockingNumber(s), getPhase(s), s['E'], s['period']))
            outfile.write('{}, {}, {}, {}, {}\n'.format(s[0], s[1], s[2], s[3], s[4]))

    with open('./bifurcations-duplicates.txt', 'w') as outfile:
        for s in duplicates:
            outfile.write('{}, {}, {}, {}, {}\n'.format(s[0], s[1], s[2], s[3], s[4]))

    m_vals = range(200)
    counts_1 = [0 for _ in range(200)]
    unique_counts_1 = [0 for _ in range(200)]
    counts_2 = [0 for _ in range(200)]
    unique_counts_2 = [0 for _ in range(200)]
    for s in bifurcations:
        m = int(s[0])
        if m > len(counts_1):
            continue
        n = int(s[1])
        phase = s[2]
        unique = is_unique(m, n)
        if phase == 1:
            counts_1[m] += 1
            if unique:
                unique_counts_1[m] += 1
        else:
            counts_2[m] += 1
            if unique:
                unique_counts_2[m] += 1
    
    counts_all = list(zip(m_vals, counts_1, counts_2, unique_counts_1, unique_counts_2))
#     print('\n'.join(counts_all))
    with open('./num-modes.csv', 'w') as outfile:
        for i in range(len(counts_1)):
            c = counts_all[i]
            if sum(c[1:]) > 0:
                outfile.write('{}, {}, {}, {}, {}\n'.format(c[0], c[1], c[2], c[3], c[4]))

    print('done')

convert()
# %timeit convert()

reading ./states/states001.txt
reading ./states/states001_1_0.txt
reading ./states/states001_8_0.txt
reading ./states/states002.txt
reading ./states/states002_8_0.txt
reading ./states/states003.txt
reading ./states/states004.txt
reading ./states/states005.txt
reading ./states/states006.txt
reading ./states/states007.txt
reading ./states/states009.txt
reading ./states/states009_198_126.txt
reading ./states/states011.txt
reading ./states/states011_66_62.txt
reading ./states/states013.txt
reading ./states/states015.txt
reading ./states/states019.txt
reading ./states/states023.txt
reading ./states/states027.txt
reading ./states/states027_166_145.txt
reading ./states/states027_187_155.txt
reading ./states/states031.txt
reading ./states/states039.txt
reading ./states/states047.txt
reading ./states/states047_262_244.txt
reading ./states/states047_302_286.txt
reading ./states/states047_334_331.txt
reading ./states/states055.txt
reading ./states/states055_242_226.txt
reading ./states/states055_